In [ ]:
import pandas as pd
from pathlib import Path

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import glob

import keras


In [ ]:
train_data = pd.read_csv('../input/train.csv')


In [ ]:
train_data.shape


In [ ]:
train_data.head()


In [ ]:
train_data.has_cactus.unique()


In [ ]:
train_data.has_cactus.hist()


In [ ]:
train_data.has_cactus.value_counts()


In [ ]:
train_data.has_cactus.plot()


In [ ]:
positive_examples = train_data[train_data.has_cactus==1]
negative_examples = train_data[train_data.has_cactus==0]


In [ ]:
img = mpimg.imread('../input/train/train/' + positive_examples.id.tolist()[2])
plt.imshow(img)


In [ ]:
img = mpimg.imread('../input/train/train/' + negative_examples.id.tolist()[2])
plt.imshow(img)


In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(32, (5, 5), activation='relu', input_shape=(32, 32, 3)))
model.add(keras.layers.Conv2D(32, (5, 5), activation='relu'))
model.add(keras.layers.Conv2D(64, (5, 5), activation='relu'))
model.add(keras.layers.Conv2D(64, (5, 5), activation='relu'))
model.add(keras.layers.Conv2D(128, (3, 3), activation='relu'))
model.add(keras.layers.Conv2D(128, (3, 3), activation='relu'))
model.add(keras.layers.Conv2D(256, (3, 3), activation='relu'))
model.add(keras.layers.Conv2D(256, (3, 3), activation='relu'))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))


In [ ]:
model.summary()


In [ ]:
opt = keras.optimizers.Adam(0.0001)
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
train_data.shape[0]


In [ ]:
# dataaug = keras.preprocessing.image.ImageDataGenerator()


In [ ]:
def image_generator(batch_size = 64, train=True):
    while True:
        if train:
            indexes = np.arange(train_data[:15000].shape[0])
        else:
            indexes = np.arange(train_data[15000:].shape[0])
        np.random.shuffle(indexes)
        N = int(len(indexes) / batch_size)
       

        # Read in each input, perform preprocessing and get labels
        for i in range(N):
            current_indexes = indexes[i*batch_size: (i+1)*batch_size]
            batch_input = []
            batch_output = [] 
            for index in current_indexes:
                img = mpimg.imread('../input/train/train/' + train_data.id[index])
                batch_input += [(img-127)/127]
                batch_output += [train_data.has_cactus[index]]
            
            batch_input = np.array( batch_input )
            batch_output = np.array( batch_output )
        
            yield( batch_input, batch_output.reshape(-1, 1) )


In [ ]:
model.fit_generator(image_generator(), steps_per_epoch= train_data.shape[0] / 64, epochs=20)


In [ ]:
model.evaluate_generator(image_generator(train=False), steps=train_data[15000:].shape[0] / 64)


In [ ]:
!ls ../input/test/test/* | wc -l


In [ ]:
test_files = os.listdir('../input/test/test/')


In [ ]:
len(test_files)


In [ ]:
batch = 40
all_out = []
for i in range(int(4000/batch)):
    images = []
    for j in range(batch):
        img = mpimg.imread('../input/test/test/'+test_files[i*batch + j])
        images += [img]
    out = model.predict(np.array(images))
    all_out += [out]


In [ ]:
all_out = np.array(all_out).reshape(-1, 1)


In [ ]:
all_out.shape


In [ ]:
sub_file = pd.DataFrame(data = {'id': test_files, 'has_cactus': all_out.reshape(-1).tolist()})


In [ ]:
sub_file.head()


In [ ]:
sub_file.to_csv('sample_submission.csv', index=False)
